In [8]:
import sqlite3

import pandas as pd

# SETUP
DB_PATH = "0_raw_databases/1000_devices_2.db"
DB_PATH_CLEANED = "1_after_cleaning_databases/1000_devices_cleaned.db"

# to get sensible output
pd.set_option('display.width', 400)
pd.set_option('display.max_columns', 10)

In [9]:
conn = sqlite3.connect(DB_PATH)
query = "SELECT session_id, attribute, value, source FROM css_attribute"
df = pd.read_sql(query, conn)
conn.close()

## Usunięcie losowych, niezwiązanych z żadnym urządzeniem danych

In [10]:
# find all session_ids that don't have User-Agent attribute
conn = sqlite3.connect(DB_PATH)
query = """
SELECT DISTINCT session_id
FROM css_attribute
WHERE session_id NOT IN (
    SELECT DISTINCT session_id
    FROM css_attribute
    WHERE attribute = 'User-Agent'
)
"""
missing_session_ids = pd.read_sql(query, conn)
conn.close()
# remove duplicates
missing_session_ids = missing_session_ids.drop_duplicates(subset=['session_id'])

# remove found session_ids from the df dataframe
for session_id in missing_session_ids['session_id']:
    df = df[df['session_id'] != session_id]

## Usunięcie urządzeń, na których nie działa @container

In [11]:
conn = sqlite3.connect(DB_PATH)
query = """
SELECT session_id, attribute, value, source
FROM css_attribute
WHERE source = 'css' and session_id IN (
        SELECT DISTINCT session_id
        FROM css_attribute
        WHERE attribute = 'User-Agent'
    )
"""
css_session_ids = pd.read_sql(query, conn)
conn.close()
# count the number of attributes for each session_id
css_session_ids_count = css_session_ids['session_id'].value_counts()

# filter the session_ids with only 2 attributes
css_session_ids_count = css_session_ids_count[css_session_ids_count == 2]

# remove duplicates
css_session_ids_count = css_session_ids_count.reset_index()

# remove found session_ids from the df dataframe
for session_id in css_session_ids_count['session_id']:
    df = df[df['session_id'] != session_id]

## Usunięciee sesji, które nie pochodzą od ładowania strony tylko z dodatkowych requestów podczas testów browserstack

In [12]:
conn = sqlite3.connect(DB_PATH)
query = """
SELECT session_id, attribute, value, source
FROM css_attribute
WHERE session_id NOT IN (
    SELECT DISTINCT session_id
    FROM css_attribute
    WHERE source = 'browserstack'
)
"""
missing_session_ids = pd.read_sql(query, conn)
conn.close()
# remove duplicates
missing_session_ids = missing_session_ids.drop_duplicates(subset=['session_id'])

# remove found session_ids from the df dataframe
for session_id in missing_session_ids['session_id']:
    df = df[df['session_id'] != session_id]

In [13]:
# check how many session_ids are left in dataframe
unique_session_ids_count = df['session_id'].drop_duplicates().shape[0]

print(f"Number of unique session_ids: {unique_session_ids_count}")

Number of unique session_ids: 932


In [14]:
# save the cleaned dataframe to the new sqlite database
conn = sqlite3.connect(DB_PATH_CLEANED)
df.to_sql('css_attribute', conn, if_exists='replace', index=False)
conn.close()